<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/intrinsic_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure to set the runtime to GPU: Runtime -> Change runtime type -> T4 GPU

You can upload your own images, then change the relevant code cells to load it and send it through the model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# install the intrinsic decomposition repo from github
#!pip install https://github.com/compphoto/Intrinsic/archive/main.zip
!git clone https://github.com/compphoto/Intrinsic
%cd Intrinsic
!pip install .

Cloning into 'Intrinsic'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 131 (delta 11), reused 3 (delta 2), pack-reused 113
Receiving objects: 100% (131/131), 28.81 MiB | 25.26 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Intrinsic
Processing /content/Intrinsic
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/MiDaS (to revision master) to /tmp/pip-install-3by0x8pl/altered-midas_331b5b624cc945308bfb601cae417ee9
  Running command git clone --filter=blob:none --quiet https://github.com/CCareaga/MiDaS /tmp/pip-install-3by0x8pl/altered-midas_331b5b624cc945308bfb601cae417ee9
  Resolved https://github.com/CCareaga/MiDaS to commit 043f3b8d9f6d2f7e740a8ac45e0745c0b5e2fd03
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/chrislib (to revision main) to /tmp/pip-install-3by0x8pl/chrislib_c0972a591ace43a886b6408f388684bb


In [3]:
import torch

# import some helper functions from chrislib (will be installed by the intrinsic repo)
from chrislib.general import show, view, uninvert
from chrislib.data_util import load_image

# import model loading and running the pipeline
from intrinsic.pipeline import run_pipeline
from intrinsic.model_util import load_models

In [5]:
# download the pretrained weights and return the model (may take a bit to download weights)
DEVICE = 'cpu'
intrinsic_model = load_models('paper_weights', device=DEVICE)

Downloading: "https://github.com/facebookresearch/WSL-Images/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /root/.cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth
100%|██████████| 340M/340M [00:06<00:00, 58.0MB/s]
/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/rwightm

In [6]:
# load an example image from the github repo
torch.hub.download_url_to_file('https://raw.githubusercontent.com/compphoto/Intrinsic/main/figures/avocado.png', 'avo.png')

100%|██████████| 2.85M/2.85M [00:00<00:00, 73.1MB/s]


In [10]:
# load the image to run through the pipeline
img = load_image('/content/avo.png')

In [9]:
from PIL import Image
import numpy as np

In [14]:
# run the image through the pipeline (use R0 resizing dicussed in the paper)
result = run_pipeline(
    intrinsic_model,
    img,
    resize_conf=0.0,
    maintain_size=True,
    linear=False,
    device=DEVICE
)

KeyboardInterrupt: 

In [13]:
# convert the inverse shading to regular shading for visualization
shd = uninvert(result['inv_shading'])
alb = result['albedo']

In [ ]:
shd3 = np.stack((shd, shd, shd), axis=-1)


In [ ]:
img_array = (shd * 255).astype(np.uint8)
img = Image.fromarray(img_array)
img.save("output_image.png")


In [ ]:
# show the result (gamma corrects the linear intrinsic components and scales to [0-1])
show([img, c, view(alb)], size=(20, 7))

In [ ]:
# show the result (gamma corrects the linear intrinsic components and scales to [0-1])
show([c, view(shd), view(alb)], size=(20, 7))

## Intrinsic Process

In [ ]:
# Copy Data from gdrive to colab
!cp source -d dest


In [ ]:
import os

def get_image_list(data_path):
  # Get all images in dataset (return a list of path of each image)
  items = os.listdir(data_path) # Each item contains 40 images
  image_list = []
  for item in items:
    f = os.path.join(data_path, item)
    imgs = os.listdir(f)
    imgs = [os.path.join(f,imgs) for i in imgs]
    image_list.extend(imgs)
  return image_list


# Run process all images in the list

In [ ]:
data_path = ''
images = get_image_list(data_path)

print('Total: {0} images'.format(len(images)))

for image in images:
    result = run_pipeline(
      intrinsic_model,
      img,
      resize_conf=0.0,
      maintain_size=True,
      linear=False,
      device=DEVICE
    )

    albedo = results['albedo']
    inv_shd = results['inv_shading']

    # compute shading from inverse shading
    shading = uninvert(inv_shd)